# Debugging & Profiling

<hr>

## `printf` debugging and logging

Logging is better than regular print statements for several reasons:

- You can log to files, sockets or even remote servers instead of standard output.
- Logging supports severity levels (such as INFO, DEBUG, WARN, ERROR), that allow you to filter the output accordingly.
- For new issues, there’s a fair chance that your logs will contain enough information to detect what is going wrong.

A sample logging `.py` script can be found [here](https://missing.csail.mit.edu/static/files/logger.py).

### Using `pyflakes` as static code analysis

Given a code snippet below:

```python
# foobar.py
import time

def foo():
    return 42

for foo in range(5): # foo is both a function and a variable
    print(foo) 
bar = 1
bar *= 0.2
time.sleep(60)
print(baz) # baz was never defined
```

`baz` is a variable that is undefined and is being referred to at the end of the program but the error will not show up until every prior step is executed. Static analysis tools like `pyflakes` can help to identify that without running the code itself.

```console
> pyflakes foobar.py

# Output
# foobar.py:6: redefinition of unused 'foo' from line 3
# foobar.py:11: undefined name 'baz'
```

Most editors and IDEs support displaying the output of these tools within the editor itself, highlighting the locations of warnings and errors. This is often called **code linting** and it can also be used to display other types of issues such as stylistic violations or insecure constructs.

****

## Profiling

Even if your code functionally behaves as you would expect, that might not be good enough if it takes all your CPU or memory in the process. Algorithms classes often teach big O notation but not how to find hot spots in your programs. Since premature optimization is the root of all evil, you should learn about profilers and monitoring tools. 

They will help you understand which parts of your program are taking most of the time and/or resources so you can focus on optimizing those parts.

### Using `time` module in Python

The most basic way is to look at the time spent on code executed.

```python
# Print the time it took your code between two points
import time, random
n = random.randint(1, 10) * 100

# Get current time
start = time.time()

# Do some work
print("Sleeping for {} ms".format(n))
time.sleep(n/1000)

# Compute time between start and now
print(time.time() - start)

# Output
# Sleeping for 500 ms
# 0.5713930130004883
```

This is wall clock time and can be misleading as this execution time can be diluted by other parallel processes that are unrelated to the code itself.

Below are the different types of time measured:

- **Real**: Wall clock elapsed time from start to finish of the program, including the time taken by other processes and time taken while blocked (e.g. waiting for I/O or network)
- **User**: Amount of time spent in the CPU running user code
- **Sys**: Amount of time spent in the CPU running kernel code

```console
# Measure time taken to go to a website and return response
# curl - client URL
# &> /dev/null ensures no return of response output

> time curl <url> &> /dev/null

real	0m0.405s
user	0m0.041s
sys	 0m0.024s
```

Here you can see that the `sys` time is a lot shorter than the real time as a lot of the time is spent waiting for the response from the system rather than the code execution on CPU itself.

### CPU Profiling

A more intuitive way of displaying profiling information is to include the time taken per line of code, which is what line profilers do.

If we used Python’s `cProfile` profiler we’d get over 2500 lines of output, and even with sorting it’d be hard to understand where the time is being spent. A quick run with `line_profiler` shows the time taken per line:

```console
> kernprof -l -v a.py
Wrote profile results to urls.py.lprof
Timer unit: 1e-06 s

Total time: 0.636188 s
File: a.py
Function: get_urls at line 5

Line #  Hits         Time  Per Hit   % Time  Line Contents
==============================================================
 5                                           @profile
 6                                           def get_urls():
 7         1     613909.0 613909.0     96.5      response = requests.get('https://missing.csail.mit.edu')
 8         1      21559.0  21559.0      3.4      s = BeautifulSoup(response.content, 'lxml')
 9         1          2.0      2.0      0.0      urls = []
10        25        685.0     27.4      0.1      for url in s.find_all('a'):
11        24         33.0      1.4      0.0          urls.append(url['href'])
```

### Memory

Similarly, we can profile code using a `memory_profiler` to check memory usage:

```console
> python -m memory_profiler example.py

Line #    Mem usage  Increment   Line Contents
==============================================
     3                           @profile
     4      5.97 MB    0.00 MB   def my_func():
     5     13.61 MB    7.64 MB       a = [1] * (10 ** 6)
     6    166.20 MB  152.59 MB       b = [2] * (2 * 10 ** 7)
     7     13.61 MB -152.59 MB       del b
     8     13.61 MB    0.00 MB       return a
```

### Visualizing with Control flow graphs

Call graphs or control flow graphs display the relationships between subroutines within a program by including functions as nodes and functions calls between them as directed edges. When coupled with profiling information such as the number of calls and time taken, call graphs can be quite useful for interpreting the flow of a program. In Python you can use the `pycallgraph` library to generate them.

```console
pycallgraph graphviz -- ./<filename>.py
```

![Call_Graph.png](https://upload.wikimedia.org/wikipedia/commons/2/2f/A_Call_Graph_generated_by_pycallgraph.png)

****

## General resource management

Probably the most popular is `htop`, which is an improved version of top. `htop` presents various statistics for the currently running processes on the system. `htop` has a myriad of options and keybinds, some useful ones are: `<F6>` to sort processes, `t` to show tree hierarchy and `h` to toggle threads. 

Allows for live interactive sorting of the statistics as well.

****

# Basic code
A `minimal, reproducible example`